# Example

In [1]:
import numpy as np
from madminer import MadMiner

## 1. Create MadMiner object

In [2]:
miner = MadMiner()

## 2. Define parameter space

In [3]:
miner.add_parameter('sminputs', 1, 'alphaEW')
miner.add_parameter('sminputs', 2, 'GF')

miner.parameters

{'GF': ('sminputs', 2), 'alphaEW': ('sminputs', 1)}

## 3. Define benchmark points

In [4]:
miner.add_benchmark(
    {'alphaEW':120., 'GF':1.17e-6},
    'apples'
)
miner.add_benchmark(
    {'alphaEW':125., 'GF':1.16e-6},
    'oranges'
)
miner.add_benchmark(
    {'alphaEW':130., 'GF':1.15e-6},
    'avocados'
)

miner.benchmarks

{'apples': {'GF': 1.17e-06, 'alphaEW': 120.0},
 'avocados': {'GF': 1.15e-06, 'alphaEW': 130.0},
 'oranges': {'GF': 1.16e-06, 'alphaEW': 125.0}}

## 4. Export parameter and reweight cards, integrating along two different scenarios

In [5]:
miner.export_cards(
    'templates/param_card_template.dat',
    'templates/reweight_card_template.dat',
    'MG_process_1'
)

In [6]:
miner.export_cards(
    'templates/param_card_template.dat',
    'templates/reweight_card_template.dat',
    'MG_process_2',
    sample_benchmark='avocados'
)